In [1]:
import os
import json
import requests

from tqdm import tqdm

from utils import async_get, get, get_client_access_token
from utils import get_song_genres, featurize_tracks, get_dates_added

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import numpy as np

In [2]:
client_id = "d71970014054446a9f65903c9fd8afdd"
client_secret = "2be6c841192c4d4b9eda0e7faf2354c3"

access_token = get_client_access_token(client_id, client_secret)
authorization_header = {"Authorization": "Bearer {}".format(access_token)}

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
access_token = "BQA6GETg5sQhn1RyVIBl7i9rVDzMeo-Y7fgxvC2rE5b9Pyk8oqjo5vtnR5DvuU9uMKdOelMoSQY7CciVRR_QwF3_WwFsXjaiO-5FR7HlJFWkSFtyLAS9YDdjAUr2HHzxTfiD0OyePBqC_c-Ms9jlaTo6-VmRledO_jqbfUEdBSsSU8M-KQ"
authorization_header = {"Authorization": "Bearer {}".format(access_token)}

In [4]:
SPOTIFY_API_BASE_URL = "https://api.spotify.com"
API_VERSION = "v1"
SPOTIFY_API_URL = "{}/{}".format(SPOTIFY_API_BASE_URL, API_VERSION)

user_profile_api_endpoint = "{}/me".format(SPOTIFY_API_URL)
user_tracks_endpoint = "{}/tracks".format(user_profile_api_endpoint)

recently_played_endpoint = "{}/player/recently-played".format(user_profile_api_endpoint)
top_tracks_endpoint = "{}/top/tracks".format(user_profile_api_endpoint)
top_artists_endpoint = "{}/top/artists".format(user_profile_api_endpoint)

In [5]:
def get_recently_played():
    limit = 50
    response = requests.get(recently_played_endpoint + "?limit={}".format(limit), headers=authorization_header)
    recently_played = response.json()
    
    track_names, track_uris, data = featurize_tracks(recently_played['items'], sp, verbose=True)
    artist_names, artist_uris, genres = get_song_genres(recently_played['items'], sp, verbose=True)

    dates = [track['played_at'] for track in recently_played['items']]

    recently_played_data = []

    num_tracks = len(recently_played['items'])
    for i in range(num_tracks):
        track_name = track_names[i]
        date_added = dates[i]
        track_uri = track_uris[i]
        track_features = data['features'][i]
        track_genres = genres[i]
        track_artists = artist_names[i]
        recently_played_data.append({ 'name' : track_name,
                                      'uri' : track_uri,
                                      'date' : date_added,
                                      'genres' : track_genres, 
                                      'artists' : track_artists,
                                    })
        for feature, feature_name in zip(track_features, data['feature_names']):
            recently_played_data[-1][feature_name] = feature
    
    return recently_played_data

In [6]:
recently_played_data = get_recently_played()
open("user_library/steven/data_recently_played.json", "w").write(json.dumps(recently_played_data, indent=4))

{'grant_type': 'client_credentials'}
https://accounts.spotify.com/api/token
{'access_token': 'BQBkiAIjli9tFepxf1TwzLckVl0D7-rEGDSei5YRsaiGGxjAWJLiXAy0QUGFT2p9iw4hC8iwHL4n_GCwcoA', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}


34170

In [9]:
def get_top_tracks(time_range):
    limit = 50

    response = requests.get(top_tracks_endpoint + "?time_range={}&limit={}".format(time_range, limit), 
                            headers=authorization_header)

    top_tracks = response.json()
    
    track_names, track_uris, data = featurize_tracks([{ 'track' : track } for track in top_tracks['items']], sp, verbose=True)
    artist_names, artist_uris, genres = get_song_genres([{ 'track' : track } for track in top_tracks['items']], sp, verbose=True)

    top_tracks_data = []

    num_tracks = len(top_tracks['items'])
    for i in range(num_tracks):
        track_name = track_names[i]
        track_uri = track_uris[i]
        track_features = data['features'][i]
        track_genres = genres[i]
        track_artists = artist_names[i]
        top_tracks_data.append({ 'name' : track_name,
                                 'uri' : track_uri,
                                 'genres' : track_genres, 
                                 'artists' : track_artists,
                               })
        for feature, feature_name in zip(track_features, data['feature_names']):
            top_tracks_data[-1][feature_name] = feature
    
    return top_tracks_data

In [10]:
top_tracks_short_data = get_top_tracks("short_term")
open("user_library/steven/data_top_tracks_short_term.json", "w").write(json.dumps(top_tracks_short_data, indent=4))

31782

In [11]:
top_tracks_medium_data = get_top_tracks("medium_term")
open("user_library/steven/data_top_tracks_medium_term.json", "w").write(json.dumps(top_tracks_medium_data, indent=4))

33510

In [12]:
top_tracks_long_data = get_top_tracks("long_term")
open("user_library/steven/data_top_tracks_long_term.json", "w").write(json.dumps(top_tracks_long_data, indent=4))

32797

In [13]:
def get_top_artists(time_range):
    limit = 50

    response = requests.get(top_artists_endpoint + "?time_range={}&limit={}".format(time_range, limit), 
                            headers=authorization_header)

    top_artists = response.json()
    
    names = [artist['name'] for artist in top_artists['items']]
    genres = [artist['genres'] for artist in top_artists['items']]
    popularities = [artist['popularity'] for artist in top_artists['items']]
    artist_uris = [artist['uri'] for artist in top_artists['items']]
    
    top_artists_data = []

    for i in range(len(top_artists['items'])):
        top_artists_data.append({ "name" : names[i],
                                  "genres" : genres[i],
                                  "popularity" : popularities[i],
                                  "uri" : artist_uris[i]
                                })
    
    return top_artists_data

In [14]:
top_artists_short_data = get_top_artists("short_term")
open("user_library/steven/data_top_artists_short_term.json", "w").write(json.dumps(top_artists_short_data, indent=4))

18124

In [15]:
top_artists_medium_data = get_top_artists("medium_term")
open("user_library/steven/data_top_artists_medium_term.json", "w").write(json.dumps(top_artists_medium_data, indent=4))

18843

In [16]:
top_artists_long_data = get_top_artists("long_term")
open("user_library/steven/data_top_artists_long_term.json", "w").write(json.dumps(top_artists_long_data, indent=4))

16522